In [ ]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Process a text
text = "When the sun set, we started our journey."
doc = nlp(text)

# Print tokens and their parts of speech
for token in doc:
    print(token.text, token.pos_)